**This code example works with raw MS Data downloaded from the MetaboLights repository (~ 1 GiB).  The corresponding data is currently under the curation process of MetaboLights. These notebooks will be updated when the data can be publicly accessed. The running time is ~ 1 min (excluding the time required to download the data) using a Personal Computer with an 8th generation Intel i5 processor and 8 GiB memory.**

# Application 1: System suitability check, and signal drift evaluation

This notebook introduces the analysis of the application #1 published in (add DOI). It shows how to work with raw data using as an example a System Suitability Check conducted in a metabolomics experiment: System Suitability Samples (SSS) were prepared using five known chemical standards:

* Alogliptin
* Phe-Phe
* Tryptophan
* LPC 18:0
* Leu-Enk

Ten SSS samples (addressed as SSS1) were consecutively run and used to build an acceptance criteria, and then compared against values obtained from the analysis of SSS samples that were analyzed before (SSS2) and after (SSS3) the study samples. This analysis is displayed in Figure 3.

A similar analysis was conducted using QC samples that were spiked with the same compounds and with Leu-13C used as internal standard, but in this case, no acceptance criteria was defined. These results are displayed in Figure S1.

In [ ]:
import tidyms as ms
import numpy as np
import bokeh as bk
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
bk.plotting.output_notebook()
sns.set_context("paper", font_scale=1.25)

## Loading data from Metabolights

In [ ]:
# small script to access the metabolights database
# from ftplib import FTP
# ftp = FTP("ftp.ebi.ac.uk")
# ftp.login()
# ftp.cwd("databases/metabolights/studies/public/, ,")

## Loading sample metadata and standards information

In [ ]:
# Here we set the raw data path and create a DataFrame with
# sample metadata
data_path = "data/cent/"

# get sample metadata
sample_list = pd.read_csv("data/sample_list.csv", index_col=0)
sample_list["sample_path"] = data_path + sample_list["id"] + ".mzML"

In [ ]:
# compounds used for SSS check, with their m/z and expected retention times
d = {"Compound": ["Leu-13C", "Trp", "Phe-Phe",
                  "Alogliptin", "LPC 18:0", "Leu-Enk"],
     "rt": [75, 129, 320, 291, 775, 372],
     "mz": [133.1056, 205.0977,313.1552,
            340.1773, 524.3716, 556.2771]}
df = pd.DataFrame(data=d)

## Feature Detection

In [ ]:
%%time
# Feature detection in all samples. Features with m/z values from
# the compounds used for SSS
roi_params = {"targeted_mz": df["mz"].values, "tolerance": 0.015}
cwt_params = {"snr": 10}
sample_path = sample_list["sample_path"].values
# proto dm contains the feature table and roi is a dictionary
# where each key is a sample name and each value is a list
# of ROI detected in each sample
roi, proto_dm = ms.detect_features(sample_path, roi_params=roi_params,
                                   cwt_params=cwt_params)

In [ ]:
# feature table
proto_dm.head(10)

In [ ]:
# plotting ROI data
roi["NZ_20200226_005"][0].plot()

## Feature correspondence

In [ ]:
# Feature correspondence of the detected features
# after performing correspondence, a new column called cluster
# label each one of the ionic species detected across samples
cluster = ms.feature_correspondence(proto_dm, 0.01, 5)
# make_data_container organizes the feature table into
# a data matrix, feature metadata and sample metadata
data = ms.make_data_container(proto_dm, cluster, sample_list)

In [ ]:
proto_dm.head()

In [ ]:
data.data_matrix.head()

In [ ]:
data.sample_metadata.head()

In [ ]:
data.feature_metadata.head()

## Remove unwanted features using the Retention times from the standards

In [ ]:
# find the compounds used in the SSS using the expected rt 
ft_to_compound = dict()
for index in df.index:
    mz = df.loc[index, "mz"]
    rt = df.loc[index, "rt"]
    compound = df.loc[index, "Compound"]
    ft = data.select_features(mz, rt)
    ft_to_compound[ft[0]] = compound

In [ ]:
# removing features that are not from the chemical standards
rm_features = (data.data_matrix.columns
               .difference(ft_to_compound.keys()))
data.remove(rm_features, "features")

In [ ]:
# add compound name and class to each feature
proto_dm["Compound"] = proto_dm["cluster"].map(ft_to_compound)
proto_dm["class"] = proto_dm["sample"].map(sample_list["class"])

## Figure 3: m/z, Rt and area dispersion for SSS samples

In [ ]:
# FIGURE 3: 
sss_mask = proto_dm["class"].isin(["SSS1", "SSS2", "SSS3"])
sss_data = proto_dm[sss_mask].copy()

# compute mean centered m/z and rt
mean_mz = sss_data["mz"].groupby(sss_data["cluster"]).mean()
mean_rt = sss_data["rt"].groupby(sss_data["cluster"]).mean()
sss_data["mean mz"] = \
    (sss_data["mz"].groupby(sss_data["cluster"])
     .apply(lambda x: x - mean_mz[x.name]))

sss_data["mean rt"] = \
    (sss_data["rt"].groupby(sss_data["cluster"])
     .apply(lambda x: x - mean_rt[x.name]))

xvars = ["mean mz", "mean rt", "area"]
g = sns.PairGrid(data=sss_data,
                 y_vars=["Compound"],
                 x_vars=xvars,
                 hue="class",
                 hue_kws={"marker": [".", "X", "D"], "size": [6, 8, 8]},
                 height=4)
g.map(sns.stripplot)

# setting plot properties
g.axes[0, 0].set_xlim(-0.01, 0.01)
g.axes[0, 2].set_xticks(np.linspace(0, 2e5, 5))
t = g.axes[0, 2].get_xticks()
t = [str(x / 100000) for x in t ]
g.axes[0, 2].set_xticklabels(t);
g.axes[0, 0].set_xlabel("Mean centered m/z")
g.axes[0, 1].set_xlabel("Mean centered Rt [s]")
g.axes[0, 2].set_xlabel("Area / $10^{5}$ [au]");

## Figure S2: m/z, Rt and area dispersion for QC samples

In [ ]:
# FIGURE S1: 

# also remove LPC 18:0 because the area has much higher values
qc_mask = proto_dm["class"].isin(["QC"]) & (proto_dm["Compound"] != "LPC 18:0")
qc_data = proto_dm[qc_mask].copy()

# compute mean centered m/z and rt
mean_mz = qc_data["mz"].groupby(qc_data["cluster"]).mean()
mean_rt = qc_data["rt"].groupby(qc_data["cluster"]).mean()
qc_data["mean_mz"] = \
    (qc_data["mz"].groupby(qc_data["cluster"])
     .apply(lambda x: x - mean_mz[x.name]))

qc_data["mean_rt"] = \
    (qc_data["rt"].groupby(qc_data["cluster"])
     .apply(lambda x: x - mean_rt[x.name]))
# sss_data = sss_data.rename(columns={"area": "Area / 10e5 [au]"})

xvars = ["mean_mz", "mean_rt", "area"]
g = sns.PairGrid(data=qc_data,
                 y_vars=["Compound"],
                 x_vars=xvars,
                 hue="class",
                 height=4)
g.map(sns.stripplot)
g.axes[0, 0].set_xlim(-0.01, 0.01)
g.axes[0, 1].set_xlim(-1, 1)
g.axes[0, 2].set_xticks(np.linspace(0, 3e5, 6))

# ticks for lpc
# g.axes[0, 2].set_xticks(np.linspace(2e6, 7e6, 6))
t = g.axes[0, 2].get_xticks()
# t = [str(x / 100000) + "e05" if x != 0 else "0" for x in t ]
t = [str(x / 100000) for x in t ]
g.axes[0, 2].set_xticklabels(t);g.axes[0, 0].set_xlabel("Mean centered m/z")
g.axes[0, 1].set_xlabel("Mean centered Rt [s]")
g.axes[0, 2].set_xlabel("Area / $10^{5}$ [au]")
# g.savefig("rt_mz_area_qc_lpc.png", dpi=300)

### Figure S1: overlapped ROI for each feature

In [ ]:
# an optional figure to show feature correspondence results
proto_dm["roi index"] = proto_dm["roi index"].astype(int)
proto_dm["peak index"] = proto_dm["peak index"].astype(int)
def plot_clustered_roi(proto_dm, roi, ft, classes, cmapper):
    fig, ax = plt.subplots(figsize=(12, 8))
    grouped = proto_dm.groupby("cluster")
    ft_group = grouped.get_group(ft)
    for ind in ft_group.index:
        c = ft_group.loc[ind, "class"]
        if c in classes:
            sample = ft_group.loc[ind, "sample"]
            roi_ind = ft_group.loc[ind, "roi index"]
            peak_ind = ft_group.loc[ind, "peak index"]
            r = roi[sample][roi_ind]
            p = roi[sample][roi_ind].peaks[peak_ind]
            ax.plot(r.rt, r.spint, color="gray", alpha=0.5)
            ax.fill_between(r.rt[p.start:p.end+1], r.spint[p.start:p.end+1],
                            alpha=0.5, color=cmapper[c])
    return fig, ax

title_str = "Alogliptin"
kft = "FT08"
fig, ax = plot_clustered_roi(proto_dm, roi, kft, ["SSS1", "SSS2", "SSS3"],
                             {"SSS1": "lightskyblue",
                               "SSS2": "darkorange",
                               "SSS3": "green"})
# fig, ax = plot_clustered_roi(proto_dm, roi, kft, ["QC"],
#                              {"QC": "lightskyblue"})
ax.set_title(title_str)
# ax.set_xlim(750, 800)
ax.set_xlabel("Rt [s]");
# plt.savefig(kft + ".png", dpi=300)